In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f

In [5]:
# Model inherit from nn.Module
class Model(nn.Module): 

    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        # how many layers
        # instantiate the inherited Module by calling its own init method
        super().__init__() # this essentially instantiates the class we are inheriting from
        self.fc1 = nn.Linear(in_features,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.out = nn.Linear(h2,out_features)

    # Input layer (4 features)--> h1 (N1 Neurons) --> h2 (N2 Neurons) --> Output (3 classes)

    def forward(self,x):
        # F.relu is activation function
        # passing features x into fc layers and then the o/p thru' the activation function
        x = f.relu(self.fc1(x)) 
        x = f.relu(self.fc2(x))
        x = self.out(x)

        return x


In [6]:
torch.manual_seed(32)
model = Model()

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(@SYS.DATASANDBOX_PATH + '4031682022/Repo/Iris.csv')

In [9]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['Species'] = le.fit_transform(df.Species.values);

In [15]:
# X = df.drop('target', axis=1)
# y = df['target']
X = df.drop('Species', axis=1)
y = df['Species']

In [16]:
X_v = X.values
y_v = y.values

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_v,y_v,test_size=0.2,random_state=33)

In [18]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

In [23]:
X_train.shape

torch.Size([120, 5])

In [19]:
# in pytorch if you use cross entropy loss we don't need to do one-hot encoding for multiclass scenario
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [24]:
y_train.shape

torch.Size([120])

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [21]:
model.parameters

<bound method Module.parameters of Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)>

In [22]:
epochs = 100
losses = []

for i in range(epochs):

  # Forward and get a prediction
  y_pred = model.forward(X_train)

  # Calculate Loss /Error
  loss = criterion(y_pred, y_train)

  losses.append(loss)

  if i%10==0:
    print(f'Epoch {i} and Loss is: {loss}')

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (120x5 and 4x8)

In [ ]:
with torch.no_grad(): # deactivates the auto gradient engine
  plt.plot(range(epochs),losses)
  plt.ylabel('CROSS ENTROPY LOSS')
  plt.xlabel('Epoch ')

In [ ]:
with torch.no_grad():
  y_eval = model.forward(X_test)
  loss = criterion(y_eval, y_test)

In [ ]:
loss

In [ ]:
correct = 0

with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)
    # print(f'{i+1}.)   {str(y_val)}   {y_test[i]}')
    print(f'{i+1}.)   {str(y_val.argmax().item())}   {y_test[i]}')

    if y_val.argmax().item() == y_test[i]:
      correct += 1

print(f'We got {correct} correct')

In [ ]:
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('model_scripted.pth') # Save

In [ ]:
model_j = torch.jit.load('model_scripted.pth')
model_j.eval()

In [ ]:
mystery_iris = torch.tensor([5.6,3.7,2.2,0.5])

In [ ]:
with torch.no_grad():
  print(model(mystery_iris))
  print(model(mystery_iris).argmax().item())

In [ ]:
with torch.no_grad():
  # print(new_model(X_test))
  # print(new_model(X_test).argmax())

  for i, data in enumerate(model(X_test)):

  # print(f'{i+1}.)   {str(y_val)}   {y_test[i]}')
    print(f'{i+1}.)   {str(data.argmax().item())}')

In [ ]:
# saves only weights and biases
torch.save(model.state_dict(),'my_iris_model.pt') 

In [ ]:
new_model = Model() # does not know W or B
new_model.load_state_dict(torch.load('my_iris_model.pt'))

In [ ]:
new_model.eval()

In [ ]:
torch.save(model,'my_iris_model.pth') 

In [ ]:
new_model = torch.load('my_iris_model.pth')

In [ ]:
new_model.eval()

In [ ]:
mystery_iris = torch.tensor([5.6,3.7,2.2,0.5])

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,7))
fig.tight_layout()

plots = [(0,1),(2,3),(0,2),(1,3)]
colors = ['b', 'r', 'g']
labels = ['Iris setosa','Iris virginica','Iris versicolor']

for i, ax in enumerate(axes.flat):
    for j in range(3):
        x = df.columns[plots[i][0]]
        y = df.columns[plots[i][1]]
        ax.scatter(df[df['species']==j][x], df[df['species']==j][y], color=colors[j])
        ax.set(xlabel=x, ylabel=y)

    # Add a plot for our mystery iris:
    ax.scatter(mystery_iris[plots[i][0]], mystery_iris[plots[i][1]], color='y')

fig.legend(labels=labels, loc=3, bbox_to_anchor=(1.0,0.85))
plt.show()

In [ ]:
with torch.no_grad():
  print(new_model(mystery_iris))
  print(new_model(mystery_iris).argmax().item())

In [ ]:
data_list = []
with torch.no_grad():
  # print(new_model(X_test))
  # print(new_model(X_test).argmax())

  for i, data in enumerate(new_model(X_test)):

  # print(f'{i+1}.)   {str(y_val)}   {y_test[i]}')
    # print(f'{i+1}.)   {str(data.argmax().item())}')
    data_list.append(data.argmax().item())
data_list

In [ ]:
data_series = pd.Series(data_list) 

In [ ]:
X_test_d = pd.DataFrame(X_test, columns=X.columns)
X_test_d

In [ ]:
pred = pd.concat([X_test_d, data_series], axis=1)

In [ ]:
pred.rename({0: 'predictions'}, axis=1, inplace=True)
pred